In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import os

: 

In [2]:
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms(True, warn_only=True)

os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

torch.manual_seed(40)
torch.cuda.manual_seed_all(40)

In [6]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'


In [7]:
device

'cuda'

In [8]:
def read_image_tensor(image_folder,transform,num_images=None):
    if num_images==None:
        num_images = len(os.listdir(image_folder))
    images = []
    for i in range(num_images):
        img = torchvision.io.read_image(os.path.join(image_folder,f"{i}.jpg")).float()
        images.append(transform(img))
    return torch.stack(images).to(device)



In [9]:
def get_labels(csv_file):
    # TODO: Return a torch tensor after reading the labels in csv_file. Convert to float().
    labels = pd.read_csv(csv_file)["label"].values
    return torch.tensor(labels).float()

In [10]:
img_size = (256,256)
base_transform = transforms.Compose(
    [transforms.Resize(img_size)
    ]
)
train_X = read_image_tensor("../animals/train/",base_transform)/256
train_Y = get_labels("../animals/train.csv")
test_X = read_image_tensor("../animals/test/",base_transform)/256
test_Y = get_labels("../animals/test.csv")

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across th

In [11]:
train_X.shape

torch.Size([1000, 3, 256, 256])

In [12]:
def train_model(model, train_loader, test_loader, num_epochs, loss_function, optimizer):
    # TODO: Make sure you read through these lines of code and understand all key lines.
    # For example: Why do you need to zero out the gradients using optimizer.zero_grad() in the for loop?
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        for i, data in enumerate(train_loader):
            inputs,labels = data
            optimizer.zero_grad()
            output = model(inputs)
            loss = loss_function(output,labels.view(output.shape))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        average_loss = total_loss/len(train_loader)

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {average_loss:.4f}")
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data in test_loader:
                inputs, labels = data
                outputs = model(inputs)
                pred = (outputs > 0.5)*1
                correct += (pred==labels.view(pred.shape)).sum()
                total += labels.size(0)
            accur = 100*correct/total
            print(f"Test Accuracy after Epoch {epoch+1}: {accur:.2f}%")

    print("Training done.")


In [13]:
# PART 1: TODO
# Write down the model description
# model = ...
# Relevant torch.nn classes you will need include nn.Sequential, nn.Conv2d, nn.MaxPool2d and so on.
from ../deep-learning/models import *

model = UNet256()

In [14]:
model.to(device)

Sequential(
  (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Flatten(start_dim=1, end_dim=-1)
  (7): Linear(in_features=131072, out_features=64, bias=True)
  (8): ReLU()
  (9): Linear(in_features=64, out_features=1, bias=True)
  (10): Sigmoid()
)

In [15]:
train_dataset = TensorDataset(train_X.to(device), train_Y.to(device))
test_dataset = TensorDataset(test_X.to(device), test_Y.to(device))
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
num_epochs = 30

loss_func = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

In [16]:
train_model(model,train_loader,test_loader,num_epochs,loss_func,optimizer)

Epoch 1/30, Loss: 1.0351
Test Accuracy after Epoch 1: 56.75%
Epoch 2/30, Loss: 0.6895
Test Accuracy after Epoch 2: 59.00%
Epoch 3/30, Loss: 0.6794
Test Accuracy after Epoch 3: 55.00%
Epoch 4/30, Loss: 0.6650
Test Accuracy after Epoch 4: 60.50%
Epoch 5/30, Loss: 0.6242
Test Accuracy after Epoch 5: 63.25%
Epoch 6/30, Loss: 0.6178
Test Accuracy after Epoch 6: 64.25%
Epoch 7/30, Loss: 0.5659
Test Accuracy after Epoch 7: 64.25%
Epoch 8/30, Loss: 0.5127
Test Accuracy after Epoch 8: 61.00%
Epoch 9/30, Loss: 0.4695
Test Accuracy after Epoch 9: 64.50%
Epoch 10/30, Loss: 0.4035
Test Accuracy after Epoch 10: 64.25%
Epoch 11/30, Loss: 0.3353
Test Accuracy after Epoch 11: 66.50%
Epoch 12/30, Loss: 0.2727
Test Accuracy after Epoch 12: 66.75%
Epoch 13/30, Loss: 0.2427
Test Accuracy after Epoch 13: 63.75%
Epoch 14/30, Loss: 0.1707
Test Accuracy after Epoch 14: 65.75%
Epoch 15/30, Loss: 0.1147
Test Accuracy after Epoch 15: 67.75%
Epoch 16/30, Loss: 0.0859
Test Accuracy after Epoch 16: 67.50%
Epoch 17/3